In [1]:
import torch
import joblib
import os
import time
from utils import *
from tqdm import tqdm_notebook as tqdm
os.listdir('./model')

['net.pkl', 'StandardScaler.pkl']

In [2]:
import numpy as np

def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# load model

In [3]:
s = time.time()
ss = joblib.load('./model/StandardScaler.pkl')
net = torch.load('./model/net.pkl')
net.eval()
print('loading time: ',time.time()-s)

loading time:  1.7776167392730713


# load data

In [4]:
df = pd.read_csv('./data/train(add_split_factor).csv',index_col=0).sample(1000).reset_index(drop=True)
print(df.shape)
df.head()

(1000, 436)


,Case Conditions_Feed Rate to C013 (Input),Case Conditions_C7+ in Light End (Input),Case Conditions_Heart Cut Prod. Rate (Input),Case Conditions_C6P- in Heart Cut (Input),Case Conditions_Feed Rate to C013 (Results),Case Conditions_C7+ in Light End (Results),Case Conditions_Heart Cut Prod. Rate (Results),Case Conditions_C6P- in Heart Cut (Results),Case Conditions_C10+ in Heart Cut (Results),Naphtha Properties_Naphtha Feed Rate,...,"Individual Component to Heavy End Split Factor_2,2,3,3-Tetramethylhexane",Individual Component to Heavy End Split Factor_3-Methylnonane,Individual Component to Heavy End Split Factor_tert-Butylcyclohexane,Individual Component to Heavy End Split Factor_n-Decane,"Individual Component to Heavy End Split Factor_1,2-Diethylbenzene",Individual Component to Heavy End Split Factor_n-Undecane,Individual Component to Heavy End Split Factor_n-Pentylbenzene,Individual Component to Heavy End Split Factor_n-Dodecane,Individual Component to Heavy End Split Factor_n-Hexylbenzene,Individual Component to Heavy End Split Factor_n-Tridecane
0,20.0,2.5,90.0,1.000,20.000000,2.499780,90.000160,1.000038,5.025563,107.600006,...,0.372995,0.403277,0.407388,0.422442,0.435764,0.459471,0.467030,0.476830,0.480600,1.0
1,20.0,10.0,100.0,0.005,20.000002,10.000338,99.999977,0.005000,1.744711,110.000008,...,0.432836,0.457206,0.461607,0.473966,0.487327,0.506326,0.513840,0.521458,0.525249,1.0
2,20.0,2.5,100.0,1.500,20.000000,2.500001,100.000160,1.500404,4.857091,123.000000,...,0.345863,0.370151,0.375182,0.387237,0.401102,0.420992,0.428924,0.437126,0.441176,1.0
3,30.0,10.0,100.0,0.700,29.999998,10.000590,100.000030,0.699965,2.780016,116.500008,...,0.403748,0.432026,0.437672,0.451279,0.466654,0.488147,0.496664,0.505288,0.509578,1.0
4,30.0,2.5,100.0,0.700,30.000002,2.500063,99.998917,0.700012,4.443406,116.999992,...,0.428794,0.468115,0.475517,0.492250,0.509606,0.536594,0.545758,0.556606,0.561114,1.0


# define columns

In [5]:
case_col = df.columns[(df.columns.str.contains('Case'))&(df.columns.str.contains('Input'))].tolist()
case_col

['Case Conditions_Feed Rate to C013 (Input)',
 'Case Conditions_C7+ in Light End (Input)',
 'Case Conditions_Heart Cut Prod. Rate (Input)',
 'Case Conditions_C6P- in Heart Cut (Input)']

In [6]:
naphtha_col = df.columns[df.columns.str.contains('Naphtha Properties')].tolist()[5:-1]
print(len(naphtha_col))
naphtha_col[0],naphtha_col[-1]

54


('Naphtha Properties_Oxygen', 'Naphtha Properties_n-Tridecane')

In [7]:
s_col = get_group_col(df,'Individual Component')

# sort s_col
temp = []
for i in [i.split('_')[1] for i in naphtha_col]:
    col1 = 'Individual Component to Light End Split Factor_{}'.format(i)
    col2 = 'Individual Component to Heart Cut Split Factor_{}'.format(i)
    col3 = 'Individual Component to Heavy End Split Factor_{}'.format(i)
    cols = [col1,col2,col3]
    temp += cols
s_col = temp
print(len(s_col))

162


In [8]:
le_col = get_group_col(df,'Light End Product')[3:-1]
hc_col = get_group_col(df,'Heart Cut Product')[4:-1]
he_col = get_group_col(df,'Heavy End Product')[3:-1]

In [9]:
f_col = 'Naphtha Properties_Naphtha Feed Rate.1'

In [10]:
f_le = get_group_col(df,'Light End Product Properties')[1]
f_hc = get_group_col(df,'Heart Cut Product Properties')[1]
f_he = get_group_col(df,'Heavy End Product Properties')[1]
print(f_le)
print(f_hc)
print(f_he)

Light End Product Properties_Light End Prod. Rate.1
Heart Cut Product Properties_Heart Cut Prod. Rate.1
Heavy End Product Properties_Heavy End Product Rate.1


In [11]:
x_col = case_col + naphtha_col
y_col = s_col
print(len(x_col))
print(len(y_col))

58
162


# tensor

In [12]:
X = torch.tensor(ss.transform(df[x_col]),dtype=torch.float).cuda()
y = torch.tensor(df[y_col].values,dtype=torch.float).cuda()

# make split factor prediction 

In [13]:
y_pred_sp = net(X).detach().cpu().numpy()
y_pred_sp = pd.DataFrame(y_pred_sp,columns=y_col)
y_pred_sp

,Individual Component to Light End Split Factor_Oxygen,Individual Component to Heart Cut Split Factor_Oxygen,Individual Component to Heavy End Split Factor_Oxygen,Individual Component to Light End Split Factor_Methane,Individual Component to Heart Cut Split Factor_Methane,Individual Component to Heavy End Split Factor_Methane,Individual Component to Light End Split Factor_Ethane,Individual Component to Heart Cut Split Factor_Ethane,Individual Component to Heavy End Split Factor_Ethane,Individual Component to Light End Split Factor_Propane,...,Individual Component to Heavy End Split Factor_n-Pentylbenzene,Individual Component to Light End Split Factor_n-Dodecane,Individual Component to Heart Cut Split Factor_n-Dodecane,Individual Component to Heavy End Split Factor_n-Dodecane,Individual Component to Light End Split Factor_n-Hexylbenzene,Individual Component to Heart Cut Split Factor_n-Hexylbenzene,Individual Component to Heavy End Split Factor_n-Hexylbenzene,Individual Component to Light End Split Factor_n-Tridecane,Individual Component to Heart Cut Split Factor_n-Tridecane,Individual Component to Heavy End Split Factor_n-Tridecane
0,0.999900,0.000055,0.000046,0.999421,0.000296,0.000283,0.999238,0.000421,0.000342,0.997949,...,0.450275,0.008377,0.532678,0.458945,0.001290,0.530415,0.468295,0.000370,0.000131,0.999499
1,0.999984,0.000009,0.000007,0.999895,0.000053,0.000052,0.999769,0.000120,0.000111,0.999276,...,0.356876,0.005542,0.619734,0.374724,0.000262,0.627643,0.372096,0.000061,0.000038,0.999901
2,0.999982,0.000012,0.000007,0.999793,0.000128,0.000079,0.999595,0.000267,0.000138,0.998441,...,0.422112,0.005341,0.565611,0.429048,0.000508,0.563655,0.435837,0.000082,0.000039,0.999879
3,0.999953,0.000024,0.000023,0.999744,0.000127,0.000129,0.999207,0.000507,0.000286,0.997221,...,0.472199,0.006499,0.500878,0.492623,0.000736,0.503711,0.495553,0.000123,0.000059,0.999819
4,0.999853,0.000074,0.000073,0.999625,0.000187,0.000188,0.999363,0.000322,0.000315,0.998339,...,0.529944,0.006323,0.457523,0.536154,0.001249,0.439439,0.559312,0.000276,0.000157,0.999567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.999929,0.000038,0.000032,0.999343,0.000318,0.000339,0.999209,0.000450,0.000341,0.997539,...,0.363016,0.008647,0.613374,0.377979,0.001067,0.616754,0.382179,0.000334,0.000131,0.999535
996,0.999653,0.000203,0.000144,0.998655,0.000785,0.000560,0.998127,0.001053,0.000820,0.995014,...,0.484585,0.009263,0.501605,0.489131,0.002318,0.488796,0.508886,0.000652,0.000376,0.998971
997,0.999979,0.000014,0.000008,0.999911,0.000057,0.000031,0.999791,0.000127,0.000082,0.999029,...,0.525222,0.002774,0.462060,0.535167,0.000432,0.451672,0.547896,0.000039,0.000020,0.999940
998,0.998601,0.000716,0.000683,0.995719,0.002151,0.002129,0.994319,0.003242,0.002439,0.985911,...,0.471432,0.014590,0.510799,0.474611,0.006133,0.503215,0.490652,0.001902,0.000981,0.997117


In [14]:
y_real_sp = df[y_col]
y_real_sp

,Individual Component to Light End Split Factor_Oxygen,Individual Component to Heart Cut Split Factor_Oxygen,Individual Component to Heavy End Split Factor_Oxygen,Individual Component to Light End Split Factor_Methane,Individual Component to Heart Cut Split Factor_Methane,Individual Component to Heavy End Split Factor_Methane,Individual Component to Light End Split Factor_Ethane,Individual Component to Heart Cut Split Factor_Ethane,Individual Component to Heavy End Split Factor_Ethane,Individual Component to Light End Split Factor_Propane,...,Individual Component to Heavy End Split Factor_n-Pentylbenzene,Individual Component to Light End Split Factor_n-Dodecane,Individual Component to Heart Cut Split Factor_n-Dodecane,Individual Component to Heavy End Split Factor_n-Dodecane,Individual Component to Light End Split Factor_n-Hexylbenzene,Individual Component to Heart Cut Split Factor_n-Hexylbenzene,Individual Component to Heavy End Split Factor_n-Hexylbenzene,Individual Component to Light End Split Factor_n-Tridecane,Individual Component to Heart Cut Split Factor_n-Tridecane,Individual Component to Heavy End Split Factor_n-Tridecane
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.989323,...,0.467030,0.0,0.523170,0.476830,0.0,0.519400,0.480600,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.991262,...,0.513840,0.0,0.478542,0.521458,0.0,0.474751,0.525249,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.989294,...,0.428924,0.0,0.562874,0.437126,0.0,0.558824,0.441176,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.986455,...,0.496664,0.0,0.494712,0.505288,0.0,0.490422,0.509578,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.988981,...,0.545758,0.0,0.443394,0.556606,0.0,0.438886,0.561114,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.993056,...,0.377791,0.0,0.614439,0.385561,0.0,0.611110,0.388890,0.0,0.0,1.0
996,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.991059,...,0.493625,0.0,0.496971,0.503029,0.0,0.493000,0.507000,0.0,0.0,1.0
997,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.989765,...,0.536175,0.0,0.454549,0.545451,0.0,0.450545,0.549455,0.0,0.0,1.0
998,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.991907,...,0.486934,0.0,0.503224,0.496776,0.0,0.499201,0.500799,0.0,0.0,1.0


# use split factor reconstrction output

In [15]:
y_real_o = df[le_col+hc_col+he_col]
y_real_o

,Light End Product Properties_Oxygen,Light End Product Properties_Methane,Light End Product Properties_Ethane,Light End Product Properties_Propane,Light End Product Properties_n-Butane,Light End Product Properties_i-Pentane,Light End Product Properties_n-Pentane,Light End Product Properties_tr2-Pentene,Light End Product Properties_Cyclopentane,Light End Product Properties_3-Methylpentane,...,"Heavy End Product Properties_2,2,3,3-Tetramethylhexane",Heavy End Product Properties_3-Methylnonane,Heavy End Product Properties_tert-Butylcyclohexane,Heavy End Product Properties_n-Decane,"Heavy End Product Properties_1,2-Diethylbenzene",Heavy End Product Properties_n-Undecane,Heavy End Product Properties_n-Pentylbenzene,Heavy End Product Properties_n-Dodecane,Heavy End Product Properties_n-Hexylbenzene,Heavy End Product Properties_n-Tridecane
0,0.0,0.0,0.0,0.563139,1.110983,8.957810,13.704480,0.217766,1.327052,17.315098,...,5.428436,13.702488,3.911614,3.555275,1.342348,0.731722,0.081434,0.011086,0.016760,0.0
1,0.0,0.0,0.0,0.052184,0.051594,0.607563,1.456187,0.000000,0.097875,0.518839,...,2.588332,6.379480,1.657489,2.070711,3.240197,0.421679,0.196430,0.007119,0.035856,0.0
2,0.0,0.0,0.0,0.426558,0.830822,6.759150,12.487739,0.162741,2.654299,23.301594,...,3.940802,9.842321,3.495679,3.546471,2.066604,0.730990,0.126561,0.014882,0.025034,0.0
3,0.0,0.0,0.0,0.482255,0.948031,7.613201,12.989666,0.179645,2.079452,20.430750,...,4.349590,10.861958,3.667970,1.714334,1.968962,0.346813,0.122422,0.007326,0.022166,0.0
4,0.0,0.0,0.0,0.290820,0.558562,4.574760,7.205972,0.102405,1.280090,18.224876,...,6.648572,16.935917,2.973188,4.467792,4.625314,0.919939,0.288950,0.014033,0.056587,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.482340,0.932094,7.621210,14.226346,0.186754,1.674684,19.689728,...,3.686791,9.213838,3.943922,2.690942,3.095025,0.552924,0.193845,0.009650,0.038935,0.0
996,0.0,0.0,0.0,0.156914,0.310199,2.509298,6.071731,0.058673,2.124683,26.707205,...,4.929372,12.377604,3.377489,7.813800,6.117352,1.601572,0.380015,0.028900,0.075732,0.0
997,0.0,0.0,0.0,0.599995,1.177199,9.571792,19.508177,0.225614,2.257369,20.664553,...,5.353667,13.398636,5.302804,5.825828,7.524353,1.186985,0.465028,0.019989,0.093967,0.0
998,0.0,0.0,0.0,0.364245,0.721002,5.854940,10.892993,0.142447,1.594858,20.242418,...,5.388319,13.597469,4.324750,4.715627,4.302413,0.966540,0.267437,0.020463,0.055010,0.0


In [16]:
y_pred_o = pd.DataFrame(index=y_real_o.index,columns=y_real_o.columns)
y_pred_o

,Light End Product Properties_Oxygen,Light End Product Properties_Methane,Light End Product Properties_Ethane,Light End Product Properties_Propane,Light End Product Properties_n-Butane,Light End Product Properties_i-Pentane,Light End Product Properties_n-Pentane,Light End Product Properties_tr2-Pentene,Light End Product Properties_Cyclopentane,Light End Product Properties_3-Methylpentane,...,"Heavy End Product Properties_2,2,3,3-Tetramethylhexane",Heavy End Product Properties_3-Methylnonane,Heavy End Product Properties_tert-Butylcyclohexane,Heavy End Product Properties_n-Decane,"Heavy End Product Properties_1,2-Diethylbenzene",Heavy End Product Properties_n-Undecane,Heavy End Product Properties_n-Pentylbenzene,Heavy End Product Properties_n-Dodecane,Heavy End Product Properties_n-Hexylbenzene,Heavy End Product Properties_n-Tridecane
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def reconstrction(df,y_pred_sp):
    y_pred_o = pd.DataFrame(index=df.index,columns=le_col+hc_col+he_col)
    for x in naphtha_col:
        c = x.split('_')[1]
        
        s_le = 'Individual Component to Light End Split Factor_{}'.format(c)
        s_hc = 'Individual Component to Heart Cut Split Factor_{}'.format(c)
        s_he = 'Individual Component to Heavy End Split Factor_{}'.format(c)
        
        o_le = 'Light End Product Properties_{}'.format(c)
        o_hc = 'Heart Cut Product Properties_{}'.format(c)
        o_he = 'Heavy End Product Properties_{}'.format(c)
        
        y_pred_o[o_le] = df[f_col] * df[x] * y_pred_sp[s_le]
        y_pred_o[o_hc] = df[f_col] * df[x] * y_pred_sp[s_hc]
        y_pred_o[o_he] = df[f_col] * df[x] * y_pred_sp[s_he]

    y_pred_o[le_col] = y_pred_o[le_col].values / y_pred_o[le_col].sum(axis=1).values.reshape(-1,1)*100
    y_pred_o[hc_col] = y_pred_o[hc_col].values / y_pred_o[hc_col].sum(axis=1).values.reshape(-1,1)*100
    y_pred_o[he_col] = y_pred_o[he_col].values / y_pred_o[he_col].sum(axis=1).values.reshape(-1,1)*100
    
    return y_pred_o

In [18]:
y_pred_o = reconstrction(df,y_pred_sp)

In [19]:
y_pred_o['SUM'] = y_pred_o.sum(axis=1)
y_real_o['SUM'] = y_real_o.sum(axis=1)

In [20]:
y_pred_o.head()

,Light End Product Properties_Oxygen,Light End Product Properties_Methane,Light End Product Properties_Ethane,Light End Product Properties_Propane,Light End Product Properties_n-Butane,Light End Product Properties_i-Pentane,Light End Product Properties_n-Pentane,Light End Product Properties_tr2-Pentene,Light End Product Properties_Cyclopentane,Light End Product Properties_3-Methylpentane,...,Heavy End Product Properties_3-Methylnonane,Heavy End Product Properties_tert-Butylcyclohexane,Heavy End Product Properties_n-Decane,"Heavy End Product Properties_1,2-Diethylbenzene",Heavy End Product Properties_n-Undecane,Heavy End Product Properties_n-Pentylbenzene,Heavy End Product Properties_n-Dodecane,Heavy End Product Properties_n-Hexylbenzene,Heavy End Product Properties_n-Tridecane,SUM
0,0.0,0.0,0.0,0.573807,1.131042,9.031912,13.849803,0.225022,1.329990,17.387589,...,13.299549,3.826018,3.466308,1.335384,0.725823,0.079920,0.010861,0.016624,0.0,300.0
1,0.0,0.0,0.0,0.095066,0.093519,1.028662,2.452614,0.000000,0.138038,0.647877,...,4.288261,1.152976,1.346965,2.282550,0.285272,0.128603,0.004823,0.023944,0.0,300.0
2,0.0,0.0,0.0,0.435467,0.848981,6.804955,12.568429,0.164346,2.668730,23.278393,...,9.318769,3.396539,3.414156,2.014949,0.711279,0.123314,0.014462,0.024485,0.0,300.0
3,0.0,0.0,0.0,0.492784,0.964271,7.625882,12.918673,0.181963,2.015212,20.313425,...,10.375506,3.558666,1.636992,1.890001,0.335247,0.115011,0.007058,0.021300,0.0,300.0
4,0.0,0.0,0.0,0.291123,0.558743,4.540154,7.153766,0.103550,1.285017,18.130589,...,16.905150,2.888389,4.305522,4.561807,0.902329,0.279578,0.013469,0.056204,0.0,300.0


In [21]:
y_real_o.head()

,Light End Product Properties_Oxygen,Light End Product Properties_Methane,Light End Product Properties_Ethane,Light End Product Properties_Propane,Light End Product Properties_n-Butane,Light End Product Properties_i-Pentane,Light End Product Properties_n-Pentane,Light End Product Properties_tr2-Pentene,Light End Product Properties_Cyclopentane,Light End Product Properties_3-Methylpentane,...,Heavy End Product Properties_3-Methylnonane,Heavy End Product Properties_tert-Butylcyclohexane,Heavy End Product Properties_n-Decane,"Heavy End Product Properties_1,2-Diethylbenzene",Heavy End Product Properties_n-Undecane,Heavy End Product Properties_n-Pentylbenzene,Heavy End Product Properties_n-Dodecane,Heavy End Product Properties_n-Hexylbenzene,Heavy End Product Properties_n-Tridecane,SUM
0,0.0,0.0,0.0,0.563139,1.110983,8.957810,13.704480,0.217766,1.327052,17.315098,...,13.702488,3.911614,3.555275,1.342348,0.731722,0.081434,0.011086,0.016760,0.0,300.000000
1,0.0,0.0,0.0,0.052184,0.051594,0.607563,1.456187,0.000000,0.097875,0.518839,...,6.379480,1.657489,2.070711,3.240197,0.421679,0.196430,0.007119,0.035856,0.0,299.999977
2,0.0,0.0,0.0,0.426558,0.830822,6.759150,12.487739,0.162741,2.654299,23.301594,...,9.842321,3.495679,3.546471,2.066604,0.730990,0.126561,0.014882,0.025034,0.0,300.000026
3,0.0,0.0,0.0,0.482255,0.948031,7.613201,12.989666,0.179645,2.079452,20.430750,...,10.861958,3.667970,1.714334,1.968962,0.346813,0.122422,0.007326,0.022166,0.0,300.000004
4,0.0,0.0,0.0,0.290820,0.558562,4.574760,7.205972,0.102405,1.280090,18.224876,...,16.935917,2.973188,4.467792,4.625314,0.919939,0.288950,0.014033,0.056587,0.0,300.000023


# metric

In [22]:
y_col = y_real_o.columns.tolist()
y_real = y_real_o
y_pred = y_pred_o
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [23]:
res = pd.DataFrame(index=y_col,columns=['MSE','MAPE'])
for i in y_col:
  res.loc[i,'MSE'] = mean_squared_error(y_real[i],y_pred[i])
  res.loc[i,'MAPE'] = mape(y_real[i],y_pred[i])
res.loc['AVG'] = res.mean(axis=0)
res

,MSE,MAPE
Light End Product Properties_Oxygen,0,NaN
Light End Product Properties_Methane,0,NaN
Light End Product Properties_Ethane,0,NaN
Light End Product Properties_Propane,0.000389231,NaN
Light End Product Properties_n-Butane,0.00137109,NaN
Light End Product Properties_i-Pentane,0.0525934,2.10926
Light End Product Properties_n-Pentane,0.166855,2.0601
Light End Product Properties_tr2-Pentene,4.5913e-05,NaN
Light End Product Properties_Cyclopentane,0.00151897,NaN
Light End Product Properties_3-Methylpentane,0.119662,1.5984
